In [58]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *

spark = sphynx.get_spark(executor_count=4, app_name='pdu_syp2', node_spec=NODE_MEDIUM)

WARN: Spark session already exists with 8 executors.
If you want to have 4 executors, please call sphynx.stop() and recreate the session.


## CONSOLE

In [20]:
start_date = '2021-09-11'
end_date = '2021-10-17'

In [21]:
device='console'
user = load_data_mart(start_date=start_date, end_date=end_date, device=device, table_name='user_master')
gcoin = load_data_mart(start_date=start_date, end_date=end_date, device=device, table_name='gcoin_use')

In [8]:
#pc 와 비교 위한 
device='pc'
user_pc = load_data_mart(start_date=start_date, end_date=end_date, device=device, table_name='user_master')
gcoin_pc = load_data_mart(start_date=start_date, end_date=end_date, device=device, table_name='gcoin_use')

In [8]:
user.count()

1059637377

In [9]:
gcoin.count()

264063

In [6]:
au = user.where('lastlogindate >= date').groupBy('date').agg(countDistinct('accountid').alias('au')).orderBy('date')
pu = gcoin.groupBy('date').agg(countDistinct('account_id').alias('pu'), sum('price').alias('price_sum')).orderBy('date')

In [9]:
au_pc = user_pc.where('lastlogindate >= date').groupBy('date').agg(countDistinct('accountid').alias('au')).orderBy('date')
pu_pc = gcoin_pc.groupBy('date').agg(countDistinct('account_id').alias('pu'), sum('price').alias('price_sum')).orderBy('date')

In [12]:
au.show()

+----------+------+
|      date|    au|
+----------+------+
|2021-09-11|161657|
|2021-09-12|171053|
|2021-09-13|149841|
|2021-09-14|149412|
|2021-09-15|155129|
|2021-09-16|133025|
|2021-09-17|151768|
|2021-09-18|167920|
|2021-09-19|173939|
|2021-09-20|155163|
|2021-09-21|151731|
|2021-09-22|149057|
|2021-09-23|147480|
|2021-09-24|146918|
|2021-09-25|159965|
|2021-09-26|166791|
|2021-09-27|146902|
|2021-09-28|144300|
|2021-09-29|144309|
|2021-09-30|142495|
+----------+------+
only showing top 20 rows



In [15]:
pu.show()

+----------+----+---------+
|      date|  pu|price_sum|
+----------+----+---------+
|2021-09-11|2001|  3284330|
|2021-09-12|2187|  3647140|
|2021-09-13|1625|  2325170|
|2021-09-14|1457|  2005960|
|2021-09-15|1566|  2793980|
|2021-09-16|9503| 18116390|
|2021-09-17|8646| 15968160|
|2021-09-18|7617| 14124660|
|2021-09-19|5897|  9276490|
|2021-09-20|3997|  6292430|
|2021-09-21|3385|  5225210|
|2021-09-22|3033|  4695110|
|2021-09-23|2886|  4156800|
|2021-09-24|2727|  4166310|
|2021-09-25|3364|  5044800|
|2021-09-26|3255|  4650440|
|2021-09-27|2400|  3587350|
|2021-09-28|2267|  3304870|
|2021-09-29|3974|  4534550|
|2021-09-30|3314|  4058180|
+----------+----+---------+
only showing top 20 rows



In [7]:
pur = au.join(pu, 'date').withColumn('pur', col('pu')/col('au')) \
                   .withColumn('ARPU', col('price_sum')/col('au')) \
                   .withColumn('ARPPU', col('price_sum')/col('pu')).orderBy('date')

In [10]:
pur_pc = au_pc.join(pu_pc, 'date').withColumn('pur', col('pu')/col('au')) \
                   .withColumn('ARPU', col('price_sum')/col('au')) \
                   .withColumn('ARPPU', col('price_sum')/col('pu')).orderBy('date')

In [18]:
pur.show()

+----------+------+----+---------+--------------------+------------------+------------------+
|      date|    au|  pu|price_sum|                 pur|              ARPU|             ARPPU|
+----------+------+----+---------+--------------------+------------------+------------------+
|2021-09-11|161657|2001|  3284330|0.012378059719034747|20.316658109453968| 1641.344327836082|
|2021-09-12|171053|2187|  3647140|0.012785510923514935|21.321695614809446|1667.6451760402379|
|2021-09-13|149841|1625|  2325170|0.010844828851916365|15.517581970221768|1430.8738461538462|
|2021-09-14|149412|1457|  2005960|0.009751559446363077| 13.42569539260568| 1376.774193548387|
|2021-09-15|155129|1566|  2793980|0.010094824307511813| 18.01068787911996|1784.1507024265645|
|2021-09-16|133025|9503| 18116390| 0.07143769968051118|136.18785942492013| 1906.386404293381|
|2021-09-17|151768|8646| 15968160| 0.05696853091560803|105.21427441884983|1846.8841082581541|
|2021-09-18|167920|7617| 14124660| 0.04536088613625536| 84.1

In [19]:
import pandas as pd

pur_pd = pur.toPandas().round(3)

In [12]:
pur_pc_pd = pur_pc.toPandas().round(3)
pur_pc_pd.to_csv('pur_pc_pd.csv', index=False)

In [21]:
pur_pd.to_csv('pur_pd.csv', index=False)

In [22]:
pur_pd

,date,au,pu,price_sum,pur,ARPU,ARPPU
0,2021-09-11,161657,2001,3284330,0.012,20.317,1641.344
1,2021-09-12,171053,2187,3647140,0.013,21.322,1667.645
2,2021-09-13,149841,1625,2325170,0.011,15.518,1430.874
3,2021-09-14,149412,1457,2005960,0.010,13.426,1376.774
4,2021-09-15,155129,1566,2793980,0.010,18.011,1784.151
5,2021-09-16,133025,9503,18116390,0.071,136.188,1906.386
6,2021-09-17,151768,8646,15968160,0.057,105.214,1846.884
7,2021-09-18,167920,7617,14124660,0.045,84.115,1854.360
8,2021-09-19,173939,5897,9276490,0.034,53.332,1573.086
9,2021-09-20,155163,3997,6292430,0.026,40.554,1574.288


In [24]:
gcoin.where("lower(product_name) like '%bundle%' ").groupby('product_name').count().orderBy('count', ascending=False).show()

+--------------------+-----+
|        product_name|count|
+--------------------+-----+
|  MOON RABBIT BUNDLE| 3915|
|B.A.S.A. BUNNY BU...| 2464|
|PCS5 ULTIMATE BUNDLE| 1212|
|  MOON SEEKER BUNDLE|  964|
|SPACE BUNNY ULTIM...|  737|
|JISOO'S BLACKPINK...|  634|
|ROSÉ'S BLACKPINK ...|  558|
|JENNIE'S BLACKPIN...|  484|
|NO STRINGS ATTACH...|  237|
|SON HEUNG-MIN'S C...|  228|
|BLACKPINK MEGA BU...|  215|
|MARQUIS DE FOX BU...|  212|
|  FANCY BERYL BUNDLE|  210|
|VIGILANTE BACKPAC...|  177|
|SPIKED SWEETNESS ...|  173|
|VIGILANTE HELMET ...|  167|
|LISA'S BLACKPINK ...|  166|
|RACY REINDEER BUNDLE|  155|
|    LONGSKR'S BUNDLE|  150|
|  STAY FROSTY BUNDLE|  142|
+--------------------+-----+
only showing top 20 rows



In [39]:
gcoin.where("lower(product_name) like '%bundle%' and price >= 3000").groupBy('product_name').count().orderBy('count', ascending=False).show(truncate=False)

+---------------------------+-----+
|product_name               |count|
+---------------------------+-----+
|PCS5 ULTIMATE BUNDLE       |1212 |
|SPACE BUNNY ULTIMATE BUNDLE|737  |
|BLACKPINK MEGA BUNDLE      |215  |
+---------------------------+-----+



In [28]:
gcoin.where("product_name like '%emote%' ").count()

0

In [23]:
daily_gcoin_pd = gcoin.groupBy('date', 'account_id').agg(sum('price').alias('gcoin_daily_use'), 
                                        min('price').alias('min_gcoin'), 
                                        avg('price').alias('avg_gcoin'), 
                                        max('price').alias('max_gcoin'),
                                        countDistinct('transaction_id').alias('trans_time')).toPandas()

In [31]:
daily_gcoin_pd.describe()

,gcoin_daily_use,min_gcoin,avg_gcoin,max_gcoin,trans_time
count,120602.000000,120602.000000,120602.000000,120602.000000,120602.000000
mean,1636.405615,628.214789,764.647735,961.184723,2.189541
std,2877.812303,654.229239,675.177852,912.394694,2.612268
min,100.000000,100.000000,100.000000,100.000000,1.000000
25%,500.000000,200.000000,275.000000,300.000000,1.000000
50%,990.000000,500.000000,595.000000,990.000000,1.000000
75%,1800.000000,990.000000,990.000000,1190.000000,2.000000
max,116400.000000,9900.000000,9900.000000,9900.000000,86.000000


In [31]:
account_cnt_500 = daily_gcoin_pd[(daily_gcoin_pd.gcoin_daily_use >= 500) & (daily_gcoin_pd.gcoin_daily_use < 1000)].groupby('date')['account_id'].count().reset_index(name='500').sort_values('date')

In [32]:
account_cnt_3000 = daily_gcoin_pd[(daily_gcoin_pd.gcoin_daily_use >= 3000) & (daily_gcoin_pd.gcoin_daily_use < 3500)].groupby('date')['account_id'].count().reset_index(name='3000').sort_values('date')

In [33]:
account_cnt_5000 = daily_gcoin_pd[daily_gcoin_pd.gcoin_daily_use >= 5500].groupby('date')['account_id'].count().reset_index(name='5000').sort_values('date')

In [34]:
a = pd.merge(account_cnt_500, account_cnt_3000, on='date')
a = pd.merge(a, account_cnt_5000, on='date')
a.head()

,date,500,3000,5000
0,2021-09-11,601,47,102
1,2021-09-12,649,31,109
2,2021-09-13,491,29,62
3,2021-09-14,421,35,49
4,2021-09-15,458,30,93


In [45]:
a['emote'] = a['500'] * 4.99
a['bundle'] = a['3000'] * 31.99
a['mega'] = a['5000'] * 59.99

In [46]:
a.head()

,date,500,3000,5000,emote,bundle,mega
0,2021-09-11,601,47,102,2998.99,1503.53,6118.98
1,2021-09-12,649,31,109,3238.51,991.69,6538.91
2,2021-09-13,491,29,62,2450.09,927.71,3719.38
3,2021-09-14,421,35,49,2100.79,1119.65,2939.51
4,2021-09-15,458,30,93,2285.42,959.70,5579.07


In [47]:
a.to_csv('console_predicted_min.csv', index=False)

In [48]:
a[['emote', 'bundle', 'mega']].sum()

emote     201446.30
bundle    119706.58
mega      357660.38
dtype: float64

In [49]:
a[['emote', 'bundle', 'mega']].sum().values.sum()

678813.26

In [43]:
account_cnt_1 = daily_gcoin_pd[(daily_gcoin_pd.gcoin_daily_use >= 500) & (daily_gcoin_pd.gcoin_daily_use < 3000)].groupby('date')['account_id'].count().reset_index(name='500').sort_values('date')
account_cnt_2 = daily_gcoin_pd[(daily_gcoin_pd.gcoin_daily_use >= 3000) & (daily_gcoin_pd.gcoin_daily_use < 5000)].groupby('date')['account_id'].count().reset_index(name='3000').sort_values('date')
account_cnt_3 = daily_gcoin_pd[daily_gcoin_pd.gcoin_daily_use >= 5000].groupby('date')['account_id'].count().reset_index(name='5000').sort_values('date')

In [44]:
b = pd.merge(account_cnt_1, account_cnt_2, on='date')
b = pd.merge(b, account_cnt_3, on='date')
b.head()

,date,500,3000,5000
0,2021-09-11,1271,102,121
1,2021-09-12,1441,97,130
2,2021-09-13,1024,62,74
3,2021-09-14,915,66,62
4,2021-09-15,994,72,104


In [50]:
b['emote'] = b['500'] * 4.99
b['bundle'] = b['3000'] * 31.99
b['mega'] = b['5000'] * 59.99

In [53]:
b.to_csv('console_predicted_max.csv', index=False)

In [52]:
b[['emote', 'bundle', 'mega']].sum()

emote     383551.36
bundle    246898.82
mega      437867.01
dtype: float64

In [51]:
b[['emote', 'bundle', 'mega']].sum().values.sum()

1068317.1900000002

In [56]:
19274896.59

19274896.59

In [54]:
bundle_prop = gcoin.groupBy('date').agg(countDistinct('transaction_id').alias('trans_cnt'),
                                        count(when(col('product_name').like('%BUNDLE%'), True)).alias('bundle_trans_cnt'),
                                        countDistinct('account_id').alias('account_cnt'),
                                        countDistinct(when(col('product_name').like('%BUNDLE%'), col('account_id'))).alias('bundle_account_cnt'),
                                        sum('price').alias('sum_price'),
                                        sum(when(col('product_name').like('%BUNDLE%'), col('price'))).alias('bundle_sum_price')
                                            ) \
                                              .withColumn('bundle_prop', round(col('bundle_trans_cnt') / col('trans_cnt'), 3)) \
                                              .withColumn('bundle_account_prop', round(col('bundle_account_cnt') / col('account_cnt'), 3)).orderBy('date') \
                                              .withColumn('bundle_price_prop', round(col('bundle_sum_price') / col('sum_price'), 3))

In [55]:
bundle_prop_pd = bundle_prop.toPandas()

In [56]:
bundle_prop_pd

,date,trans_cnt,bundle_trans_cnt,account_cnt,bundle_account_cnt,sum_price,bundle_sum_price,bundle_prop,bundle_account_prop,bundle_price_prop
0,2021-09-11,4781,228,2001,200,3284330,627310,0.048,0.100,0.191
1,2021-09-12,5296,245,2187,220,3647140,644230,0.046,0.101,0.177
2,2021-09-13,3642,160,1625,138,2325170,401630,0.044,0.085,0.173
3,2021-09-14,3279,147,1457,127,2005960,317910,0.045,0.087,0.158
4,2021-09-15,3994,201,1566,152,2793980,515400,0.050,0.097,0.184
5,2021-09-16,17968,574,9503,559,18116390,2069760,0.032,0.059,0.114
6,2021-09-17,16372,1477,8646,1360,15968160,3523900,0.090,0.157,0.221
7,2021-09-18,16488,1283,7617,1178,14124660,2867210,0.078,0.155,0.203
8,2021-09-19,11990,856,5897,795,9276490,1772760,0.071,0.135,0.191
9,2021-09-20,8109,557,3997,506,6292430,1201420,0.069,0.127,0.191


In [58]:
bundle_prop_pd.to_csv('bundle_prop_pd.csv', index=False)

## GCOIN_USE PC

In [57]:
gcoin_pc = load_data_mart(start_date=start_date, end_date=end_date, device='pc', table_name='gcoin_use')
bundle_prop_pc = gcoin_pc.groupBy('date').agg(countDistinct('transaction_id').alias('trans_cnt'),
                                        count(when(col('product_name').like('%BUNDLE%'), True)).alias('bundle_trans_cnt'),
                                        countDistinct('account_id').alias('account_cnt'),
                                        countDistinct(when(col('product_name').like('%BUNDLE%'), col('account_id'))).alias('bundle_account_cnt'),
                                        sum('price').alias('sum_price'),
                                        sum(when(col('product_name').like('%BUNDLE%'), col('price'))).alias('bundle_sum_price')
                                            ) \
                                              .withColumn('bundle_prop', round(col('bundle_trans_cnt') / col('trans_cnt'), 3)) \
                                              .withColumn('bundle_account_prop', round(col('bundle_account_cnt') / col('account_cnt'), 3)).orderBy('date') \
                                              .withColumn('bundle_price_prop', round(col('bundle_sum_price') / col('sum_price'), 3))

In [59]:
bundle_prop_pc_pd = bundle_prop_pc.toPandas()

In [60]:
bundle_prop_pc_pd.to_csv('bundle_prop_pc_pd.csv', index=False)

In [61]:
bundle_prop_pc_pd

,date,trans_cnt,bundle_trans_cnt,account_cnt,bundle_account_cnt,sum_price,bundle_sum_price,bundle_prop,bundle_account_prop,bundle_price_prop
0,2021-09-11,152318,4372,57807,3990,150629860,15518660,0.029,0.069,0.103
1,2021-09-12,116823,3541,47211,3265,109972260,11959970,0.030,0.069,0.109
2,2021-09-13,91987,2722,35031,2491,86833530,8810450,0.030,0.071,0.101
3,2021-09-14,81547,2393,31086,2171,76685590,7723980,0.029,0.070,0.101
4,2021-09-15,90530,2367,33523,2191,81993200,8180770,0.026,0.065,0.100
5,2021-09-16,85116,2319,32263,2131,76695590,7976670,0.027,0.066,0.104
6,2021-09-17,178584,37363,80685,35438,236702050,112402470,0.209,0.439,0.475
7,2021-09-18,130769,19062,56726,17935,146771550,51924650,0.146,0.316,0.354
8,2021-09-19,114381,13262,48977,12431,118942230,35212680,0.116,0.254,0.296
9,2021-09-20,102357,10251,42711,9626,102398620,26969320,0.100,0.225,0.263
